# 🚀 WhisperX API Server - Zero Cost Meeting Transcription

**Hướng dẫn sử dụng:**
1. Chọn Runtime > Change runtime type > GPU (T4)
2. Chạy tất cả cells (Runtime > Run all)
3. App sẽ tự động kết nối (nếu mở từ link Smart Connect)
4. Nếu không tự kết nối, Copy URL ngrok và dán vào App

In [ ]:
# Cell 1: Setup & Auto-Connect Logic
import time
from google.colab import output
import IPython

SESSION_ID = None

def set_session_id(id_str):
    global SESSION_ID
    SESSION_ID = id_str
    print(f"🔗 Auto-Connect Enabled! Session ID: {SESSION_ID}")

output.register_callback('notebook.set_session_id', set_session_id)

js_code = """
try {
    var hash = window.location.hash;
    if (hash && hash.includes('session=')) {
        var id = hash.split('session=')[1].split('&')[0];
        google.colab.kernel.invokeFunction('notebook.set_session_id', [id], {});
        document.body.insertAdjacentHTML('beforebegin', 
            '<div style="background:#dcfce7;color:#166534;padding:10px;text-align:center;font-weight:bold;font-size:16px">' +
            '✅ Connected to Akari App (Session: ' + id + ')' +
            '</div>');
    }
} catch (e) { console.error('Auto-connect error:', e); }
"""
display(IPython.display.Javascript(js_code))
time.sleep(2) # Wait for JS

In [ ]:
# Cell 2: Install Dependencies (Fixed)
!pip install whisperx flask flask-cors pyngrok python-dotenv requests -q
!pip install faster-whisper -q

# Fix Torch Version Mismatch (Pin to Stable 2.3.1)
!pip uninstall torch torchvision torchaudio -y
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu118
print("✅ Cài đặt hoàn tất!")

In [ ]:
# Cell 3: Configuration
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import whisperx
import gc
import torch
import os
from threading import Thread

# ⚠️ CẤU HÌNH TOKEN (Chỉ cần làm 1 lần)
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN"  
HF_TOKEN = "YOUR_HF_TOKEN" 

app = Flask(__name__)
CORS(app)
port = 5000

In [ ]:
# Cell 4: Load Model (Smart Load)
import gc

# 1. Cleanup Memory First
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 16

# --- MONKEY PATCH FIX FOR TORCH 2.6+ ---
try:
    _original_torch_load = torch.load
    def safe_torch_load(*args, **kwargs):
        if 'weights_only' not in kwargs:
            kwargs['weights_only'] = False
        return _original_torch_load(*args, **kwargs)
    torch.load = safe_torch_load
    print("🔧 Applied 'weights_only=False' patch for Torch load")
except Exception as e:
    print(f"⚠️ Could not patch torch.load: {e}")
# ---------------------------------------

print(f"🎯 Device: {device}")

try:
    # Try loading with float16 (Best Quality)
    compute_type = "float16" if device == "cuda" else "int8"
    print("📦 Đang load WhisperX model (large-v3) [float16]...")
    model = whisperx.load_model("large-v3", device, compute_type=compute_type)
    print("✅ Model loaded successfully (float16)!")
    
except Exception as e:
    error_msg = str(e)
    if "out of memory" in error_msg or "CUDA out of memory" in error_msg:
        print("⚠️ Hết bộ nhớ VRAM với float16. Đang chuyển sang int8 (Nhẹ hơn)...")
        # Cleanup again
        gc.collect()
        torch.cuda.empty_cache()
        
        # Fallback to int8
        compute_type = "int8"
        model = whisperx.load_model("large-v3", device, compute_type=compute_type)
        print("✅ Model loaded successfully (int8)!")
    else:
        # Re-raise other errors
        raise e

In [ ]:
# Cell 5: API Controller
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "healthy", "device": device, "compute_type": compute_type})

@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    try:
        if 'file' not in request.files:
            return jsonify({"error": "No file uploaded"}), 400
        
        file = request.files['file']
        if file.filename == '':
            return jsonify({"error": "Empty filename"}), 400
        
        # Check language param
        language = request.form.get('language')
        
        # Save temp file
        temp_path = "/tmp/temp_audio.wav"
        file.save(temp_path)
        
        print(f"📝 Processing: {file.filename} (Lang: {language})")
        
        # 1. Load audio
        audio = whisperx.load_audio(temp_path)
        
        # 2. Transcribe
        options = {"batch_size": batch_size}
        if language and language not in ['auto', '']:
            options["language"] = language

        result = model.transcribe(audio, **options)
        
        # 3. Align
        if result["segments"]:
            model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
            result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)
        
        # 4. Diarization
        if HF_TOKEN and HF_TOKEN != "YOUR_HF_TOKEN":
            try:
                diarize_model = whisperx.DiarizationPipeline(use_auth_token=HF_TOKEN, device=device)
                diarize_segments = diarize_model(audio)
                result = whisperx.assign_word_speakers(diarize_segments, result)
            except Exception as e:
                print(f"⚠️ Diarization error: {e}")
        
        # Cleanup
        gc.collect(); torch.cuda.empty_cache()
        if os.path.exists(temp_path): os.remove(temp_path)
        
        return jsonify({"success": True, "segments": result["segments"]})
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return jsonify({"error": str(e)}), 500

In [ ]:
# Cell 6: Run Server & Sync
import requests

if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTH_TOKEN":
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    public_url = ngrok.connect(port)
    print(f"\n🚀 Server Running at: {public_url}")
    
    # AUTO-SYNC logic (Disabled to prevent errors)
    # if SESSION_ID:
    #     try:
    #         requests.post(f"https://dweet.io/dweet/for/{SESSION_ID}", json={"url": public_url, "status": "ready"})
    #     except Exception as e: pass
else:
    print("⚠️ Missing Ngrok Token. Please update Cell 3.")

app.run(port=port, debug=False, use_reloader=False)